# Langchain crash course

In [1]:
import os
from pprint import pprint
import json

In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

## LLMs

In [3]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")

## Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

## Chains

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
results = chain.invoke("Mexican")
print(json.dumps(results, indent=4))

In [ ]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [ ]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

results = chain.invoke("Indian")
print(results["input"])
pprint(results["output"])

#### Sequential Chain

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [ ]:
results = chain.invoke({"cuisine": "Indian"})
pprint(results["cuisine"])
pprint(results["restaurant_name"])
pprint(results["menu_items"])

## Agents

In [ ]:
SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY')

#### serpapi and llm-math tool

In [ ]:
from langchain import hub
from langchain.agents import AgentType, create_react_agent, load_tools, AgentExecutor

In [ ]:
prompt = hub.pull("hwchase17/react")
prompt

In [ ]:
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "What was the GDP of US in 2022?"

# Let's test it out!
agent_executor.invoke(input={"input": query})

#### Wikipedia and llm-math tool

In [ ]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query = "When was Elon musk born? What is the square root of his age as of 2023?"

# Let's test it out!
agent_executor.invoke(input={"input": query})

In [ ]:
7.211102550927978 ** 2

## Memory

#### ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.invoke("Mexican")
print(name)

In [ ]:
name = chain.invoke("Arabic")
print(name)

In [ ]:
print(chain.memory.buffer)

#### ConversationChain

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

In [ ]:
convo.invoke("Who won the first cricket world cup?")

In [ ]:
convo.invoke("How much is 5+5?")

In [ ]:
convo.invoke("Who was the captain ofthe winning team?")

In [ ]:
print(convo.memory.buffer)

#### ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.invoke("Who won the first cricket world cup?")

In [ ]:
convo.invoke("How much is 5+5?")

In [ ]:
convo.invoke("Who was the captain of the winning team?")

## Streaming

In [ ]:
for chunk in convo.stream("Write me a song about goldfish on the moon"):
    print(chunk["history"])
    # print(chunk.content, end="", flush=True)

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import GPT4All

# Set up the prompt template
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# Set up the local instace of the LLM
local_path = "/Users/brianroepke/Library/Application Support/nomic.ai/GPT4All/mistral-7b-openorca.Q4_0.gguf"  # noqa: E501
callbacks = [StreamingStdOutCallbackHandler()]
memory = ConversationBufferMemory()
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True, streaming=True)  # noqa: E501
# llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

llm_chain = ConversationChain(llm=llm)

In [ ]:
for chunk in llm_chain.stream("Write me a song about sparkling water."):
    print(chunk, end="", flush=True)